In [15]:
import numpy as np 
import torch
from data.wrappers import load_data
from model.core.odegpvae import ODEGPVAE

# device = 'cuda'
data_root = 'data/'
task = 'mnist'


In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
from model.core.dsvpy import DSVGP_Layer

In [38]:
gp = DSVGP_Layer(D_in=8*2, D_out=9, #2q, q
                M=50,
                S=100)

In [41]:
next(gp.parameters()).device.type

'cpu'

In [ ]:
@property
def device(self):
    return next(self.parameters()).device.type

In [26]:
from torch.distributions import MultivariateNormal, Normal

In [ ]:
mvn_q = MultivariateNormal(torch.zeros(8).to(device),torch.eye(q).to(device))

In [16]:
########### data ############ 
trainset, testset = load_data(data_root, task, plot=True)


########### model ###########
#odegpvae = ODEGPVAE(q=8,n_filt=16, device=device).to(device)

OSError: could not read bytes

In [3]:
len(trainset)

20

In [52]:
next(odegpvae.ode_model.parameters()).is_cuda

True

In [53]:
next(odegpvae.parameters()).is_cuda

True

In [54]:
next(odegpvae.enc_s.parameters()).is_cuda

True

In [55]:
next(odegpvae.decoder.parameters()).is_cuda

True

In [56]:
next(odegpvae.enc_v.parameters()).is_cuda

True

In [57]:
odegpvae.ode_model.svgp.device

device(type='cuda', index=0)

In [3]:
# ########### train ###########
Nepoch = 500
optimizer = torch.optim.Adam(odegpvae.parameters(),lr=1e-3)

In [39]:
minibatch = next(iter(trainset)).to(device)

In [40]:
X = minibatch

In [30]:
[N,T,nc,d,d] = X.shape

In [31]:
X.shape

torch.Size([25, 16, 1, 28, 28])

In [8]:
X[:,0].shape

torch.Size([25, 1, 28, 28])

In [9]:
#h = odegpvae.encoder(X[:,0])
s0_mu, s0_logv = odegpvae.enc_s(X[:,0])

In [10]:
v0_mu, v0_logv = odegpvae.enc_v(torch.squeeze(X[:,0:odegpvae.v_steps]))

In [66]:
print(s0_mu.shape, s0_logv.shape)
print(v0_mu.shape, v0_logv.shape)

torch.Size([25, 8]) torch.Size([25, 8])
torch.Size([25, 8]) torch.Size([25, 8])


In [11]:
eps_s0   = torch.randn_like(s0_mu)
eps_v0   = torch.randn_like(v0_mu)

In [68]:
eps_v0.is_cuda

True

In [12]:
s0 = s0_mu + eps_s0*torch.exp(s0_logv) #N,q
v0 = v0_mu + eps_v0*torch.exp(v0_logv) #N,q

In [70]:
odegpvae._zero_mean.is_cuda

True

In [14]:
logp0 = odegpvae.mvn_q.log_prob(eps_s0) + odegpvae.mvn_q.log_prob(eps_v0) # N 

In [15]:
logp0.shape

torch.Size([25])

ode

In [15]:
z0 = torch.concat([v0,s0],dim=1)

In [73]:
z0.shape

torch.Size([25, 16])

In [17]:
odegpvae.ode_model.svgp.Lzz.evaluate().is_cuda

True

In [16]:
odegpvae.ode_model.svgp.variational_strategy.variational_distribution.scale_tril.is_cuda

True

In [18]:
gp_draw = odegpvae.ode_model.svgp.draw_posterior_function()
oderhs = lambda t, x: odegpvae.ode_model.svgp.ode_rhs(t,x,gp_draw)

In [19]:
from torchdiffeq import odeint
ts  =  0.1 * torch.arange(T,dtype=torch.float).to(z0.device)
zt, logp = odeint(oderhs, (z0, logp0), ts, method="euler")

In [20]:
ztL   = []
logpL = []
ztL.append(zt.permute([1,0,2]).unsqueeze(0)) # 1,N,T,2q
logpL.append(logp.permute([1,0]).unsqueeze(0)) # 1,N,T
ztL   = torch.cat(ztL,0)
logpL = torch.cat(logpL)

decode

In [31]:
st_muL = ztL[:,:,:,8:] # L,N,T,q Only the position is decoded
s = odegpvae.fc3(st_muL.contiguous().view([1*N*T,8]) ) # L*N*T,h_dim
Xrec = odegpvae.decoder(s) # L*N*T,nc,d,d
Xrec = Xrec.view([1,N,T,nc,d,d]) # L,N,T,nc,d,d

In [32]:
Xrec.shape

torch.Size([1, 25, 16, 1, 28, 28])

Check data

In [43]:
import scipy.io as sio

In [42]:
fullname = os.path.join('data/', "rot_mnist", "rot-mnist.mat")

In [44]:
dataset = sio.loadmat(fullname)

In [45]:
X = np.squeeze(dataset['X'])

In [47]:
for key in dataset:
    print(key)

__header__
__version__
__globals__
X
Y


In [48]:
dataset['Y']

array([[[0.],
        [7.],
        [1.],
        ...,
        [4.],
        [3.],
        [2.]]])

In [54]:
Ys = np.squeeze(dataset['Y'])

In [75]:
mask = Ys[]

In [55]:
X_masked = np.ma.masked_where(Ys==3,X)

IndexError: Inconsistent shape between the condition and the input (got (10000,) and (10000, 16, 784))

In [60]:
X[Ys==9,:,:].shape

(982, 16, 784)

In [61]:
X

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.08260946, 0.08260946, 0.08260946, ..., 0.08260946,
         0.08260946, 0.08260946],
        [0.07876532, 0.07876532, 0.07876532, ..., 0.07876532,
         0.07876532, 0.07876532],
        ...,
        [0.080609  , 0.080609  , 0.080609  , ..., 0.080609  ,
         0.080609  , 0.080609  ],
        [0.08547502, 0.08547502, 0.08547502, ..., 0.08547502,
         0.08547502, 0.08547502],
        [0.00048756, 0.00048756, 0.00048756, ..., 0.00048756,
         0.00048756, 0.00048756]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.06745703, 0.06745703, 0.06745703, ..., 0.06745703,
         0.06745703, 0.06745703],
        [0.07662455, 0.07662455, 0.07662455, ..., 0.07662455,
         0.07662455, 0.07662455],
        ...,
        [0.08402812, 0.08402812, 0.08402812, ..., 0.08402812,
         0.08402812, 0.08402812],
        [0.0